In [1]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import concatenate
from tensorflow.keras.losses import binary_crossentropy
import tensorflow as tf
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import re
import random
from random import randint
from nltk.stem import PorterStemmer
porter = PorterStemmer()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import json
import re
import nltk
from nltk.tokenize import sent_tokenize
import numpy as np
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import pandas  as pd

In [5]:
passage_ids=pd.read_csv("/content/drive/MyDrive/passage_ids.csv")
df=pd.read_csv("/content/drive/MyDrive/Squad_PreProcessed.csv")

In [6]:
passage_ids.set_index("id",inplace=True)

In [7]:
def getsent(x):
    sent=sent_tokenize(passage_ids.loc[x].values[0])
    return str(sent)


In [8]:
df["sentences"]=df["passage_id"].apply(lambda x : getsent(x))

In [25]:
def prepare_dataset(df):
    req=df[["question","sentences","CorrectSentence"]]
    vals=req.values
    ma=0
    for i in range(len(vals)):
        se=eval(vals[i][1])
        ma=max(len(se),ma)
    maxpad=ma
    print(maxpad)
    sents=[]
    for i in range(len(vals)):
        se=[vals[i][0]]+eval(vals[i][1])
        se+=[""]*(maxpad+1-len(se))
        sents.append(se)
    vocab=[]
    def tokenize_data(sentences):
        for i in range(len(sentences)):
            for j in range(len(sentences[i])):
                sentences[i][j]=sentences[i][j].replace("."," . ")
                sentences[i][j]=nltk.word_tokenize(sentences[i][j])
                sentences[i][j]=list(map(lambda x :porter.stem(x),sentences[i][j]))
                sentences[i][j]=list(map(lambda x :re.sub('[^A-Za-z0-9\. ]+', '',x),sentences[i][j]))
                sentences[i][j]=list(map(lambda x :re.sub('[^A-Za-z0-9\. ]+', '',x),sentences[i][j]))
                sentences[i][j]=list(filter(lambda x: x != "" and x != ".", sentences[i][j]))

                vocab.extend(sentences[i][j])

            print(i,"***")


        return sentences,vocab

    sentences,vocab=tokenize_data(sents[:10000])
    return sentences,vocab


sentences,vocab=prepare_dataset(df)

Streaming output truncated to the last 5000 lines.
5000 ***
5001 ***
5002 ***
5003 ***
5004 ***
5005 ***
5006 ***
5007 ***
5008 ***
5009 ***
5010 ***
5011 ***
5012 ***
5013 ***
5014 ***
5015 ***
5016 ***
5017 ***
5018 ***
5019 ***
5020 ***
5021 ***
5022 ***
5023 ***
5024 ***
5025 ***
5026 ***
5027 ***
5028 ***
5029 ***
5030 ***
5031 ***
5032 ***
5033 ***
5034 ***
5035 ***
5036 ***
5037 ***
5038 ***
5039 ***
5040 ***
5041 ***
5042 ***
5043 ***
5044 ***
5045 ***
5046 ***
5047 ***
5048 ***
5049 ***
5050 ***
5051 ***
5052 ***
5053 ***
5054 ***
5055 ***
5056 ***
5057 ***
5058 ***
5059 ***
5060 ***
5061 ***
5062 ***
5063 ***
5064 ***
5065 ***
5066 ***
5067 ***
5068 ***
5069 ***
5070 ***
5071 ***
5072 ***
5073 ***
5074 ***
5075 ***
5076 ***
5077 ***
5078 ***
5079 ***
5080 ***
5081 ***
5082 ***
5083 ***
5084 ***
5085 ***
5086 ***
5087 ***
5088 ***
5089 ***
5090 ***
5091 ***
5092 ***
5093 ***
5094 ***
5095 ***
5096 ***
5097 ***
5098 ***
5099 ***
5100 ***
5101 ***
5102 ***
5103 ***
5104 ***
5105

In [26]:
temp=sentences

In [27]:
vocab=list(set(vocab))

In [28]:
def embedding_for_vocab(filepath, word_index,
                        embedding_dim):
    vocab_size = len(word_index) + 1

    # Adding again 1 because of reserved 0 index
    embedding_matrix_vocab = np.zeros((vocab_size,
                                       embedding_dim))

    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix_vocab[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix_vocab


In [29]:
from tensorflow.keras.preprocessing.text import Tokenizer

# create the dict.
tokenizer = Tokenizer()
tokenizer.fit_on_texts(vocab)

In [30]:
embedding_dim = 100

embedding_matrix_vocab = embedding_for_vocab(
    '/content/drive/MyDrive/glove/glove.6B.100d.txt', tokenizer.word_index,
  embedding_dim)



In [36]:
print("Dense vector for first word is => ",
      embedding_matrix_vocab[5])

Dense vector for first word is =>  [-1.89219996e-01 -6.67249978e-01 -1.04970000e-04  3.66289988e-02
 -8.56480002e-01  1.64590001e-01  9.99400020e-01 -1.23989999e+00
 -2.65509993e-01  1.68099999e-01 -6.03309989e-01 -3.39110009e-02
 -3.17019999e-01 -8.37180018e-01  1.56039998e-01  7.22949982e-01
  7.99269974e-01 -3.81300002e-01 -1.37390003e-01  4.61860001e-02
 -1.02919996e+00 -4.33229990e-02  5.77920020e-01 -1.77420005e-01
  6.41260028e-01 -3.49770010e-01  1.49470001e-01 -5.58000028e-01
 -9.39690024e-02  4.20590006e-02 -2.54779994e-01 -2.47740000e-01
 -7.36159980e-01  5.90040028e-01 -7.05469996e-02  4.15499985e-01
 -3.68820012e-01 -5.02269983e-01 -7.34210014e-02 -1.54330003e+00
  5.55549979e-01  7.87230015e-01  4.36890006e-01  1.36539996e+00
  5.16340017e-01 -6.40770018e-01 -6.31240010e-01  7.30289996e-01
 -8.82350028e-01  5.57319999e-01  3.10099989e-01  2.89050013e-01
  1.01709998e+00 -4.30429995e-01 -2.17309996e-01  2.27890000e-01
 -8.37440014e-01 -8.59249979e-02 -4.63389993e-01 -4.962

In [37]:
maxp=0
for j in range(len(sentences)):
    sentence=sentences[j]
    for i in range(len(sentence)):
        maxp=max(maxp,len(sentence[i]))

In [38]:
maxp

107

In [39]:
sentences=temp

In [40]:
print(len(sentences[0]))

28


In [41]:
labels=df["CorrectSentence"].values
labels=labels[:10000]

In [43]:

v_index={}
for i in range(len(vocab)):
    v_index.update({vocab[i]:i})

fin_arr = np.zeros((10000*28,2,maxp, 100))
ylabels=np.zeros((10000*28))
iter=0
for j in range(len(sentences)):
    sentence=sentences[j]
    for i in range(len(sentence)):
        for word in range(len(sentence[i])):
          fin_arr[iter,1,word]=embedding_matrix_vocab[tokenizer.word_index[sentence[i][word]]]

        if(labels[j]==i):
          ylabels[iter]=1

        for word in range(len(sentence[0])):
          fin_arr[iter,0,word]=embedding_matrix_vocab[tokenizer.word_index[sentence[0][word]]]

        iter+=1


    print(j)



Streaming output truncated to the last 5000 lines.
5000
5001
5002
5003
5004
5005
5006
5007
5008
5009
5010
5011
5012
5013
5014
5015
5016
5017
5018
5019
5020
5021
5022
5023
5024
5025
5026
5027
5028
5029
5030
5031
5032
5033
5034
5035
5036
5037
5038
5039
5040
5041
5042
5043
5044
5045
5046
5047
5048
5049
5050
5051
5052
5053
5054
5055
5056
5057
5058
5059
5060
5061
5062
5063
5064
5065
5066
5067
5068
5069
5070
5071
5072
5073
5074
5075
5076
5077
5078
5079
5080
5081
5082
5083
5084
5085
5086
5087
5088
5089
5090
5091
5092
5093
5094
5095
5096
5097
5098
5099
5100
5101
5102
5103
5104
5105
5106
5107
5108
5109
5110
5111
5112
5113
5114
5115
5116
5117
5118
5119
5120
5121
5122
5123
5124
5125
5126
5127
5128
5129
5130
5131
5132
5133
5134
5135
5136
5137
5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189

In [45]:
# model = Sequential()
inp=Input(shape=fin_arr[0].shape)
flat=Flatten()(inp)
l1=Dense(64, activation='relu')(flat)
l2=Dense(64, activation='relu')(l1)
fin=Dense(1,activation="sigmoid")(l2)
model=Model(inputs=inp,outputs=fin)

In [24]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2, 107, 50)]      0         
                                                                 
 flatten (Flatten)           (None, 10700)             0         
                                                                 
 dense (Dense)               (None, 64)                684864    
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 689089 (2.63 MB)
Trainable params: 689089 (2.63 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
def generate_batch(X = fin_arr, y = ylabels, batch_size = 200):
    ''' Generate a batch of data '''
    while True:
      for j in range(0, len(X), batch_size):
        yield(X[j:j+batch_size],ylabels[j:j+batch_size])
    # pass

In [46]:
batch_size = 100
epochs = 20
sep=int(0.9*len(fin_arr))
xtrain,xtest,ytrain,ytest=fin_arr[:sep],fin_arr[sep:],ylabels[:sep],ylabels[sep:]
# history=model.fit_generator(generator = generate_batch(xtrain, ytrain, batch_size = batch_size),
#                     steps_per_epoch =len(xtrain)//batch_size,
#                     epochs=epochs,
#                     validation_data = generate_batch(xtest, ytest, batch_size = batch_size),
#                     validation_steps = len(ytest)//batch_size)

In [62]:
len(ylabels),len(fin_arr)

(280000, 280000)

In [54]:
lbs=model.predict(xtest[:1000])
act=ytest[:1000]
acc=0
s=0
for i in range(len(lbs)):
  if(lbs[i]>=0.5):
    ans=1
  else:
    ans=0
  s+=ans
  if(ans==act[i] and act[i]==1):
    acc+=1
print(acc,np.sum(act),s)

32/32 [==============================] - 0s 10ms/step
11 38.0 39


In [42]:
def getrep(text):
  sentences[i][j]=sentences[i][j].replace("."," . ")
  text=nltk.word_tokenize(sentences[i][j])
  sentences[i][j]=list(map(lambda x :porter.stem(x),sentences[i][j]))
  sentences[i][j]=list(map(lambda x :re.sub('[^A-Za-z0-9\. ]+', '',x),sentences[i][j]))
  sentences[i][j]=list(map(lambda x :re.sub('[^A-Za-z0-9\. ]+', '',x),sentences[i][j]))
  sentences[i][j]=list(filter(lambda x: x != "" and x != ".", sentences[i][j]))



array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

In [50]:
from sklearn.linear_model import LogisticRegression
m = LogisticRegression()
xt=xtrain
yt=ytrain
print(xt.shape)
xt=xt.reshape(xt.shape[0],xt.shape[1]*xt.shape[2]*xt.shape[3])

m.fit(xt, ytrain)

(27000, 2, 384)


IndexError: ignored

In [42]:
xt2=xtest
yt2=ytest
xt2=xt2.reshape(xt2.shape[0],xt2.shape[1]*xt2.shape[2])
pred=m.predict_proba(xt2)

In [43]:
pred.shape

(243000, 2)

In [44]:
ydum=ytest.reshape(ytest.shape[0]//27,27)

In [46]:
pred=pred.reshape(pred.shape[0]//27,27,2)

In [ ]:
acc=0
for i in range(len(pred)):
  probs=pred[i]
  ma=0
  ind=0
  for j in range(len(probs)):
    if(probs[j][1]>ma):
      ma=probs[j][1]
      ind=j
  print(ind,"*******",np.argmax(ydum[i]))
  if(np.argmax(ydum[i])==ind):
    acc+=1

print(acc,len(pred))

In [10]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=a7501f84c6a7b612ddcd7fa6087e40e68532053b0d8c5fe125686758c0ef69bd
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [133]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [134]:
# embedding = model.encode(sentence)
def cosine_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)

    if norm_vector1 == 0 or norm_vector2 == 0:
        return 0  # To handle zero vectors

    similarity = dot_product / (norm_vector1 * norm_vector2)
    return similarity

In [135]:
sentences=[]

In [137]:
def prepare_dataset(df):
    req=df[["question","sentences","CorrectSentence"]]
    vals=req.values
    ma=0
    for i in range(len(vals)):
        se=eval(vals[i][1])
        ma=max(len(se),ma)
    dataset=np.zeros((10000,ma,2,384))
    qns=model.encode(vals[:10000,0])
    for i in range(len(dataset)):
      se=eval(vals[i][1])
      dataset[i,:len(se),1]=model.encode(se)
      dataset[i,:,0]=qns[i]
      print(i)
    return dataset



sentences=prepare_dataset(df)

Streaming output truncated to the last 5000 lines.
5000
5001
5002
5003
5004
5005
5006
5007
5008
5009
5010
5011
5012
5013
5014
5015
5016
5017
5018
5019
5020
5021
5022
5023
5024
5025
5026
5027
5028
5029
5030
5031
5032
5033
5034
5035
5036
5037
5038
5039
5040
5041
5042
5043
5044
5045
5046
5047
5048
5049
5050
5051
5052
5053
5054
5055
5056
5057
5058
5059
5060
5061
5062
5063
5064
5065
5066
5067
5068
5069
5070
5071
5072
5073
5074
5075
5076
5077
5078
5079
5080
5081
5082
5083
5084
5085
5086
5087
5088
5089
5090
5091
5092
5093
5094
5095
5096
5097
5098
5099
5100
5101
5102
5103
5104
5105
5106
5107
5108
5109
5110
5111
5112
5113
5114
5115
5116
5117
5118
5119
5120
5121
5122
5123
5124
5125
5126
5127
5128
5129
5130
5131
5132
5133
5134
5135
5136
5137
5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189

In [139]:
# train_x=np.zeros((4*sentences.shape[0],sentences.shape[2],sentences.shape[3]))
# train_y=np.zeros(4*sentences.shape[0])
# labels=df["CorrectSentence"].values[:10000]
# iter=0

# import random
# for i in range(len(sentences)):
#   integer_set = list(range(8))
#   if(i in integer_set):
#     integer_set.remove(labels[i])
#   act=[labels[i]]+random.sample(integer_set,3)
#   train_x[iter:iter+4]=sentences[i][act]
#   train_y[iter:iter+4]=np.array([1,0,0,0])
#   iter+=4


In [140]:
train_x[-1].shape

(2, 384)

In [151]:
x=sentences.reshape(sentences.shape[0]*sentences.shape[1],sentences.shape[2],sentences.shape[3])

In [22]:
labels=df["CorrectSentence"].values[:10000]

In [152]:
ylabels=np.zeros(sentences.shape[0]*sentences.shape[1])
iter=0
for i in range(len(sentences)):
  for j in range(len(sentences[i])):
    if(labels[i]==j):
      ylabels[iter]=1

    iter+=1

In [153]:
inp=Input(shape=x[0].shape)
flat=Flatten()(inp)
l1=Dense(64, activation='relu')(flat)
l2=Dense(64, activation='relu')(l1)
fin=Dense(1,activation="sigmoid")(l2)
model=Model(inputs=inp,outputs=fin)

In [154]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 2, 384)]          0         
                                                                 
 flatten_2 (Flatten)         (None, 768)               0         
                                                                 
 dense_6 (Dense)             (None, 64)                49216     
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 53441 (208.75 KB)
Trainable params: 53441 (208.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [155]:
def generate_batch(X = x, y = ylabels, batch_size = 200):
    ''' Generate a batch of data '''
    while True:
      for j in range(0, len(X), batch_size):
        yield(X[j:j+batch_size],ylabels[j:j+batch_size])
    # pass

In [156]:
batch_size = 100
epochs = 20
sep=int(0.1*len(x))
xtrain,ytrain,xtest,ytest=x[:sep],x[sep:],ylabels[:sep],ylabels[sep:]
history=model.fit_generator(generator = generate_batch(xtrain, ytrain, batch_size = batch_size),
                    steps_per_epoch =len(xtrain)//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(xtest, ytest, batch_size = batch_size),
                    validation_steps = len(ytest)//batch_size)

Epoch 1/20


<ipython-input-156-6b18be353821>:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(generator = generate_batch(xtrain, ytrain, batch_size = batch_size),


266/270 [============================>.] - ETA: 0s - loss: 0.1281 - accuracy: 0.9585

ValueError: ignored

In [145]:
xt2=xtest
yt2=ytest
# xt2=xt2.reshape(xt2.shape[0],xt2.shape[1]*xt2.shape[2])
pred=model.predict(xt2)

7594/7594 [==============================] - 13s 2ms/step


In [146]:
pred=pred.reshape(pred.shape[0]//27,27,1)

In [148]:
np.argmax(pred[0])

5

In [149]:
ydum=ytest.reshape(ytest.shape[0]//27,27)

In [150]:
acc=0
for i in range(len(pred)):
  probs=pred[i]
  ind=np.argmax(probs)

  if(np.argmax(ydum[i])==ind):
    acc+=1

print(acc,len(pred))

1429 9000
